# MODELO LDA

El modelo LDA (Latent Dirichlet Allocation) es una técnica de modelado de temas en procesamiento de lenguaje natural. Su objetivo es descubrir temas subyacentes en un conjunto de textos.

Los tópicos son conjuntos de palabras que frecuentemente aparecen juntas en los documentos. Cada tópico representa un tema o concepto común en el texto, esto ayuda a identificar temas recurrentes (como "calidad del servicio" o "ambiente acogedor") mencioandos en reseñas.

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
from gensim import corpora, models
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
# Carga de dataframe
df_review = pd.read_parquet('../review_final_final.parquet')
df_review

In [ ]:
# Seleccionar en df_positive solo reseñas positivas
df_positive = df_review[df_review['sentiment'] == 'positive']


In [ ]:
df_positive['tokens'] = df_positive['text'].apply(lambda x: x.lower().split())
# Crear el diccionario y el corpus
dictionary = corpora.Dictionary(df_positive['tokens'])
corpus = [dictionary.doc2bow(text) for text in df_positive['tokens']]

In [ ]:
# Aplicar LDA
num_topics = 15
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

In [ ]:
# Visualizar el modelo LDA
lda_visualization = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(lda_visualization, 'lda_visualization.html')

In [ ]:
# Imprimir los tópicos
for idx, topic in lda_model.print_topics(num_words=10):
    print(f"Topic {idx}: {topic}")

Creacion de diccionario en base a cada topico

In [ ]:
topic_mapping2 = {
    0: "Tiempo de servicio o entrega optimo",
    1: "Buena comida y relación calidad-precio",
    2: "Calidad del servicio al cliente por parte del staff'",
    3: "Opciones de platos específicos y comidas saludables",
    4: "Fidelización de clientes",
    5: "Porciones y precios razonables",
    6: "Variedad de desayunos y opciones de café",

    7: "Platos siempre frescos y bien preparados",
    8: "Ambiente familiar y agradable",
    9: "Comida auténtica y selección variada",
    10: "Buena experiencia gastronómica",
    
    11: "Comodidad y experiencia hogareña",

    12: "Servicio excelente y rápido",

    13: "Platos populares, simples y funcionales",
    14: " ",
    15: ' '
}


In [ ]:
# Obtener los tópicos principales para cada reseña
def get_top_topics(lda_model, bow, top_n=2):
    topics = sorted(lda_model[bow], key=lambda x: -x[1])
    return topics[:top_n]

In [ ]:
# Aplicar la función para obtener los tópicos principales en cada reseña
df_positive['top_topics'] = df_positive['tokens'].apply(
    lambda x: get_top_topics(lda_model, dictionary.doc2bow(x))
)

In [ ]:
# Separar los dos tópicos principales en columnas diferentes
df_positive['topic_1'] = df_positive['top_topics'].apply(lambda x: x[0][0] if len(x) > 0 else None)

# Convertir a int
df_positive['topic_1'] = df_positive['topic_1'].astype(int)

# Aplicar el mapeo usando el diccionario
df_positive['topic_1'] = df_positive['topic_1'].map(topic_mapping2) 

In [ ]:
df_business = pd.read_csv('../restaurantes_california.csv')
df_business.drop(columns=['Unnamed: 0' , 'cluster','primary_category'], inplace=True)
df_business.head()

In [ ]:
# Agrupar por gmap_id y contar la frecuencia de topic_1
topic_counts = df_positive.groupby(['gmap_id', 'topic_1']).size().reset_index(name='count')

# Obtener el tópico más frecuente para cada gmap_id
top_topic = topic_counts.loc[topic_counts.groupby('gmap_id')['count'].idxmax()]

In [ ]:
# Renombrar la columna topic_1 a top_topic_1 para diferenciar en el merge
top_topic = top_topic.rename(columns={'topic_1': 'top_topic_1'})


# Hacer el merge con df_business usando gmap_id
df_merged = pd.merge(df_business, top_topic[['gmap_id', 'top_topic_1']], on='gmap_id', how='left')
df_merged.head(10)


In [ ]:
# incluir el segundo tópico más repetido:
topic_counts_sorted = topic_counts.sort_values(by=['gmap_id', 'count'], ascending=[True, False])
second_top_topic = topic_counts_sorted.groupby('gmap_id').nth(1).reset_index()

# Renombrar la columna topic_1 a second_top_topic para diferenciar en el merge
second_top_topic = second_top_topic.rename(columns={'topic_1': 'second_top_topic'})

# Hacer el merge con df_business para incluir el segundo tópico más repetido
df_merged = pd.merge(df_merged, second_top_topic[['gmap_id', 'second_top_topic']], on='gmap_id', how='left')

In [ ]:
# Reemplazar nulos
df_merged['top_topic_1'].fillna('Establecimiento sin reseñas o rating insuficiente', inplace=True)
df_merged['second_top_topic'].fillna('', inplace=True)

# Unir en una sola columna
df_merged['caracteristicas_clave'] = df_merged['top_topic_1'].astype(str) + ". " + df_merged['second_top_topic'].astype(str)

# Eliminar columnas innecesarias
df_merged.drop(columns=['top_topic_1','second_top_topic'], inplace = True)

In [ ]:
# Guardar datos
#df_merged.to_parquet('3-caracteristicas.parquet', index=False)